In [1]:
import mysql.connector

In [2]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    passwd = "********",
    database = "math_curves"
)

In [3]:
cursor = mydb.cursor(buffered = True)
#buffered = True means cursor will get all the results from the database but
#only show those requested

In [ ]:
#Create a database for mathematical curves
cursor.execute("CREATE DATABASE math_curves")

In [4]:
#Show databases
cursor.execute("SHOW DATABASES")
for db in cursor:
    print(db[0])

experiment
information_schema
math_curves
mysql
performance_schema
sakila
testdb
world


cursor.execute("SHOW DATABASES")
for db in cursor:
    print(db[0])

In [40]:
#Creating a table in math_curves
cursor.execute('''
    CREATE TABLE simple_curves (
    value_id INT NOT NULL AUTO_INCREMENT,
    x_value INT,
    line INT,
    parabola_up INT,
    parabbola_down INT,
    cubic INT,
    reciprocal INT,
    PRIMARY KEY (value_id)
    )
''')

In [41]:
#changing the data type of the reciprocal column to "float"
cursor.execute('''
    ALTER TABLE simple_curves
    MODIFY COLUMN reciprocal FLOAT;
''')
mydb.commit()

In [42]:
#changing the column name 'parabbola_down' to 'parabola_down'
cursor.execute('''
    ALTER TABLE simple_curves
    RENAME COLUMN parabbola_down TO parabola_down;
''')
mydb.commit()

In [43]:
#Inserting values into the simple_curves table quickly with a for loop (values from -500 to 500)
for i in range(-501,500):
    if i+1 == 0: #preventing error from division by zero
        cursor.execute(f'''
            INSERT INTO simple_curves (x_value, line, parabola_up, parabola_down, cubic, reciprocal)
            VALUES ({(i+1)},{(i+1)},{(i+1)**2},{-((i+1)**2)},{(i+1)**3},DEFAULT)
        ''')
    else:
        cursor.execute(f'''
            INSERT INTO simple_curves (x_value, line, parabola_up, parabola_down, cubic, reciprocal)
            VALUES ({(i+1)},{(i+1)},{(i+1)**2},{-((i+1)**2)},{(i+1)**3},{1/(i+1)})
        ''')
mydb.commit()

In [48]:
#Import more necessary libraries for data processing and visualization
import pandas as pd
import plotly
import plotly.express as px

In [6]:
all_data = "SELECT * FROM simple_curves;"
df = pd.read_sql(all_data, mydb)

In [7]:
df.head()

,value_id,x_value,line,parabola_up,parabola_down,cubic,reciprocal
0,1,-500,-500,250000,-250000,-125000000,-0.002000
1,2,-499,-499,249001,-249001,-124251499,-0.002004
2,3,-498,-498,248004,-248004,-123505992,-0.002008
3,4,-497,-497,247009,-247009,-122763473,-0.002012
4,5,-496,-496,246016,-246016,-122023936,-0.002016


In [8]:
#Drop the default index and set the value_id column as the new index (not necessary)

df = df.set_index('value_id')

In [9]:
df.head()

,x_value,line,parabola_up,parabola_down,cubic,reciprocal
value_id,,,,,,
1,-500,-500,250000,-250000,-125000000,-0.002000
2,-499,-499,249001,-249001,-124251499,-0.002004
3,-498,-498,248004,-248004,-123505992,-0.002008
4,-497,-497,247009,-247009,-122763473,-0.002012
5,-496,-496,246016,-246016,-122023936,-0.002016


In [44]:
#Plotting graphs with Plotly

fig = px.line(df, x = 'x_value', y = ['line', 'parabola_up', 'parabola_down', 'cubic','reciprocal'])
fig.update_xaxes(range=[-100, 100])
fig.update_yaxes(range=[-500, 500])
fig.show()

In [112]:
#Create a new table for the horizontal hyperbolic curve with a and b = 5

cursor.execute('''
    CREATE TABLE hyperbola_h (
    x_value INT,
    hyperbola_h FLOAT
    )
''')

In [11]:
#Show the newly added table

cursor.execute("SHOW TABLES")

for tables in cursor:
    print(tables[0])

hyperbola_h
hyperbola_v
simple_curves


In [114]:
#Because the horizontal hyperbola will have both positive and negative y values,
#we have to rename the column of the table to 'hyper_h_positive' and add another column called 'hyper_h_negative'

#To rename the colun:
cursor.execute('''
    ALTER TABLE hyperbola_h
    RENAME COLUMN hyperbola_h TO hyper_h_positive;
''')
mydb.commit()

In [115]:
#Add the hyper_h_negative column to the hyperbola_h table:

cursor.execute('''
    ALTER TABLE hyperbola_h
    ADD COLUMN hyper_h_negative FLOAT AFTER hyper_h_positive;
''')
mydb.commit()

In [12]:
import math #To use the sqrt()

In [116]:
#Inserting values into the hyperbola_h table
for i in range(-501, 500):
    if (i+1)**2 < 10000:
        cursor.execute(f'''
            INSERT INTO hyperbola_h (x_value, hyper_h_positive, hyper_h_negative)
            VALUES ({i+1}, DEFAULT, DEFAULT)
        ''')
    else:
        cursor.execute(f'''
            INSERT INTO hyperbola_h (x_value, hyper_h_positive, hyper_h_negative)
            VALUES ({i+1}, {math.sqrt(((i+1)**2)-10000)}, {-math.sqrt(((i+1)**2)-10000)})
        ''')
mydb.commit()

In [13]:
#Try plottiing the hyperbolar function alone first to see if we have correct values
#Preparing the data

hyperbola_h_data = "SELECT * FROM hyperbola_h"
df_hyperbola_h = pd.read_sql(hyperbola_h_data, mydb)

In [45]:
#Plotting

fig_hyperbola_h = px.line(df_hyperbola_h, x = 'x_value', y = ['hyper_h_positive', 'hyper_h_negative'])
fig_hyperbola_h.update_xaxes(range=[-500, 500])
fig_hyperbola_h.update_yaxes(range=[-500, 500])

for trace in fig_hyperbola_h['data']:
    if trace['name'] == 'hyper_h_positive':
        trace['name'] = 'horizontal hyperbola'
        trace['line']['color'] = 'blue'
    
    if trace['name'] == 'hyper_h_negative':
        trace['showlegend'] = False
        trace['line']['color'] = 'blue'

fig_hyperbola_h.show()

In [140]:
#Creating another table for a vertical hyperbola equation

cursor.execute('''
    CREATE TABLE hyperbola_v (
    x_value INT,
    hyper_v_positive FLOAT,
    hhyper_v_negative FLOAT
    )
''')
mydb.commit()

In [141]:
#Changing the misspelled column name:

cursor.execute('''
    ALTER TABLE hyperbola_v
    RENAME COLUMN hhyper_v_negative TO hyper_v_negative;
''')
mydb.commit()

In [15]:
#Show the newly added table

cursor.execute('''
    SHOW TABLES;
''')

for tables in cursor:
    print(tables[0])

hyperbola_h
hyperbola_v
simple_curves


In [143]:
#Adding values to the new hyperbola_v table:

for i in range(-501, 500):
        cursor.execute(f'''
            INSERT INTO hyperbola_v (x_value, hyper_v_positive, hyper_v_negative)
            VALUES ({i+1}, {math.sqrt(10000+((i+1)**2))}, {-(math.sqrt(10000+((i+1)**2)))})
        ''')
mydb.commit()

In [16]:
#preparing data

hyperbola_v_data = "SELECT * FROM hyperbola_v"

df_hyperbola_v = pd.read_sql(hyperbola_v_data,mydb)

In [46]:
#Plotting the vertical hyperbola

fig_hyperbola_v = px.line(df_hyperbola_v, x = 'x_value', y = ['hyper_v_positive', 'hyper_v_negative'])
fig_hyperbola_v.update_xaxes(range=[-500, 500])
fig_hyperbola_v.update_yaxes(range=[-500, 500])

for trace in fig_hyperbola_v['data']:
    if trace['name'] == 'hyper_v_positive':
        trace['name'] = 'vertical hyperbola'
        trace['line']['color'] = 'red'
    
    if trace['name'] == 'hyper_v_negative':
        trace['showlegend'] = False
        trace['line']['color'] = 'red'

fig_hyperbola_v.show()

In [147]:
#Joining all the tables together to visualize everything together
#check all the tables now:
cursor.execute('''
    SHOW TABLES;
''')

for tables in cursor:
    print(tables[0])

hyperbola_h
hyperbola_v
simple_curves


In [18]:
#Joining simple_curves,hyperbola_h,hyperbola_v on 'x_value'

all_data = '''
    SELECT * FROM simple_curves
    JOIN hyperbola_h
    ON simple_curves.x_value = hyperbola_h.x_value
    JOIN hyperbola_v
    ON hyperbola_h.x_value = hyperbola_v.x_value;
'''

all_data_df = pd.read_sql(all_data,mydb)

In [19]:
all_data_df.head()

,value_id,x_value,line,parabola_up,parabola_down,cubic,reciprocal,x_value,hyper_h_positive,hyper_h_negative,x_value,hyper_v_positive,hyper_v_negative
0,1,-500,-500,250000,-250000,-125000000,-0.002000,-500,489.898,-489.898,-500,509.902,-509.902
1,2,-499,-499,249001,-249001,-124251499,-0.002004,-499,488.877,-488.877,-499,508.921,-508.921
2,3,-498,-498,248004,-248004,-123505992,-0.002008,-498,487.857,-487.857,-498,507.941,-507.941
3,4,-497,-497,247009,-247009,-122763473,-0.002012,-497,486.836,-486.836,-497,506.961,-506.961
4,5,-496,-496,246016,-246016,-122023936,-0.002016,-496,485.815,-485.815,-496,505.980,-505.980


In [20]:
#There are multiple columns with the name 'x_value'
#dropping any of them with all_data_df.drop(all_data_df.coumns[index], inplace = True, axis = 1) will only drop all of them
#We must use the following technique described here: https://bit.ly/355PwdQ to remove duplicate columns

all_data_df = all_data_df.iloc[:,~all_data_df.columns.duplicated()]
all_data_df.head()

,value_id,x_value,line,parabola_up,parabola_down,cubic,reciprocal,hyper_h_positive,hyper_h_negative,hyper_v_positive,hyper_v_negative
0,1,-500,-500,250000,-250000,-125000000,-0.002000,489.898,-489.898,509.902,-509.902
1,2,-499,-499,249001,-249001,-124251499,-0.002004,488.877,-488.877,508.921,-508.921
2,3,-498,-498,248004,-248004,-123505992,-0.002008,487.857,-487.857,507.941,-507.941
3,4,-497,-497,247009,-247009,-122763473,-0.002012,486.836,-486.836,506.961,-506.961
4,5,-496,-496,246016,-246016,-122023936,-0.002016,485.815,-485.815,505.980,-505.980


In [50]:
#plotting everything together:
all_fig = px.line(all_data_df, x = 'x_value', y = ['line', 'parabola_up', 'parabola_down', 'cubic', 'hyper_h_positive',
                                                  'hyper_h_negative','hyper_v_positive', 'hyper_v_negative'],
                 labels = dict(x_value = "x", value = "y", variable = "equations"))
all_fig.update_xaxes(range=[-150, 150])
all_fig.update_yaxes(range=[-150, 150],showticksuffix='first')
all_fig.update_layout(plot_bgcolor="black")
all_fig.update_layout(paper_bgcolor="black")
all_fig.update_layout(font_color="white")

for trace in all_fig['data']:
    if trace['name'] == 'line':
        trace['name'] = 'linear'
        trace['line']['color'] = 'yellow'
    
    if trace['name'] == 'parabola_up':
        trace['name'] = 'upward parabola'
        trace['line']['color'] = 'orchid'
        
    if trace['name'] == 'parabola_down':
        trace['name'] = 'downward parabola'
    
    if trace['name'] == 'hyper_h_positive':
        trace['name'] = 'horizontal hyperbola'
        trace['line']['color'] = 'blue'
    
    if trace['name'] == 'hyper_h_negative':
        trace['showlegend'] = False
        trace['line']['color'] = 'blue'

    if trace['name'] == 'hyper_v_positive':
        trace['name'] = 'vertical hyperbola'
        trace['line']['color'] = 'red'
    
    if trace['name'] == 'hyper_v_negative':
        trace['showlegend'] = False
        trace['line']['color'] = 'red'
    
    
all_fig.show()

In [ ]:
mydb.close()